In [ ]:
#required installs
!pip install yahoo_fin
!pip install py_vollib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 5.3 MB/s eta 0:00:00
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62855 sha256=adea705a81a972643962e8fc642d675ff684ce9c3b540093312521374a22436e
  Stored in directory: /root/.cache/pip/wheels/b9/a9/77/5836fef7a93c50d71258c47ab174ed66b12205a53574dd16c8
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24465 sha256=d26a317b65910aa95950462b8ad9ab20085a6f391090b184745fb9014a3e9fdf
  Stored in directory: /root/.cache/pip/wheels/86/91/7a/e16c79c61e339ee18d69d7eeeea89adb1d93e878139e64ace3
Successfully built py_vollib py_lets_be_rational


In [ ]:
#required imports
import numpy as np
from scipy.stats import binom
import math
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from yahoo_fin import options as op
import numpy as np
import pandas as pd
import datetime
from scipy.stats import norm
from scipy.optimize import newton
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility

Pick 3 stocks of your choice from the market.

In [ ]:
# Define start and end dates and tickers
end_date = '2023-04-28'
start_date = '2022-04-27'
tickers = ["AAPL", "AMZN", "TSLA"]

# Download stock data
stocks = yf.download(tickers, start=start_date, end=end_date)


#stock price
# Remove rows with missing data
stocks_clean = stocks['Adj Close'].dropna()

#visualize data
print("Stock Price:\n", stocks_clean)


#option price
#get expiration dates
#April 28th expiry
print("Put Option")
print("Stock : {}".format("AAPL"))
experiration_dates = op.get_expiration_dates("AAPL")

#put
put_option_data_AAPL = op.get_puts("AAPL", date=experiration_dates[0])
print(put_option_data_AAPL)

print("Stock : {}".format("AMZN"))
experiration_dates = op.get_expiration_dates("AMZN")

#put
put_option_data_AMZN = op.get_puts("AMZN", date=experiration_dates[0])
print(put_option_data_AMZN)

print("Stock : {}".format("TSLA"))
experiration_dates = op.get_expiration_dates("TSLA")

#put
put_option_data_TSLA = op.get_puts("TSLA", date=experiration_dates[0])
print(put_option_data_TSLA)

#May 5th expiry
print("Put Option")
print("Stock : {}".format("AAPL"))
# experiration_dates = op.get_expiration_dates("AAPL", "05/05/23")

#put
put_option_data_AAPL_2 = op.get_puts("AAPL", "05/05/23")
print(put_option_data_AAPL_2)

print("Stock : {}".format("AMZN"))
# experiration_dates = op.get_expiration_dates("AMZN", "05/05/23")

#put
put_option_data_AMZN_2 = op.get_puts("AMZN", "05/05/23")
print(put_option_data_AMZN_2)

print("Stock : {}".format("TSLA"))
# experiration_dates = op.get_expiration_dates("TSLA", "05/05/23")

#put
put_option_data_TSLA_2 = op.get_puts("TSLA", "05/05/23")
print(put_option_data_TSLA_2)

#May 12th expiry
print("Put Option")
print("Stock : {}".format("AAPL"))
# experiration_dates = op.get_expiration_dates("AAPL", "12/05/23")

#put
put_option_data_AAPL_3 = op.get_puts("AAPL", "05/12/23")
print(put_option_data_AAPL_3)

print("Stock : {}".format("AMZN"))
# experiration_dates = op.get_expiration_dates("AMZN", "12/05/23")

#put
put_option_data_AMZN_3 = op.get_puts("AMZN", "05/12/23")
print(put_option_data_AMZN_3)

print("Stock : {}".format("TSLA"))
# experiration_dates = op.get_expiration_dates("TSLA", "12/05/23")

#put
put_option_data_TSLA_3 = op.get_puts("TSLA", "05/12/23")
print(put_option_data_TSLA_3)

[*********************100%***********************]  3 of 3 completed
Stock Price:
                   AAPL        AMZN        TSLA
Date                                          
2022-04-27  155.627274  138.167007  293.836670
2022-04-28  162.654694  144.596497  292.503326
2022-04-29  156.700729  124.281502  290.253326
2022-05-02  157.008896  124.500000  300.980011
2022-05-03  158.519714  124.253502  303.083344
...                ...         ...         ...
2023-04-21  165.020004  106.959999  165.080002
2023-04-24  165.330002  106.209999  162.550003
2023-04-25  163.770004  102.570000  160.669998
2023-04-26  163.759995  104.980003  153.750000
2023-04-27  166.789993  109.980103  157.399994

[252 rows x 3 columns]
Put Option
Stock : AAPL
          Contract Name         Last Trade Date  Strike  Last Price    Bid  \
0   AAPL230428P00050000  2023-04-26 10:15AM EDT    50.0        0.01   0.00   
1   AAPL230428P00060000   2023-04-26 9:49AM EDT    60.0        0.01   0.00   
2   AAPL230428P00080000 

Models

In [ ]:
#binomial model
def binomial_model(S, K, T, r, sigma, N, option_type):
    """
    This function calculates the price of an option using the binomial model.

    Parameters:
        S (float): current stock price
        K (float): strike price
        T (float): time to maturity (in years)
        r (float): risk-free rate
        sigma (float): volatility
        N (int): number of time steps
        option_type (str): "European" or "American"

    Returns:
        option_price (float): price of the option
    """

    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    stock_price = np.zeros((N+1, N+1))
    option_value = np.zeros((N+1, N+1))

    for i in range(N+1):
        for j in range(i+1):
            stock_price[j, i] = S * (u ** (i-j)) * (d ** j)

    if option_type == "European":
        option_value[:, N] = np.maximum(0, K - stock_price[:, N])
    elif option_type == "American":
        option_value[:, N] = np.maximum(0, K - stock_price[:, N])

    for i in range(N-1, -1, -1):
        for j in range(i+1):
            if option_type == "European":
                option_value[j, i] = np.exp(-r * dt) * (p * option_value[j, i+1] + (1-p) * option_value[j+1, i+1])
            elif option_type == "American":
                exercise_value = K - stock_price[j, i]
                option_value[j, i] = max(exercise_value, np.exp(-r * dt) * (p * option_value[j, i+1] + (1-p) * option_value[j+1, i+1]))

    stock_price = stock_price[0, 0]
    option_price = option_value[0, 0]

    return option_price, stock_price


#black scholes model
def black_scholes(S, K, T, r, sigma, option_type):
    """
    This function calculates the price of an option using the Black-Scholes formula.

    Parameters:
        S (float): current stock price
        K (float): strike price
        T (float): time to maturity (in years)
        r (float): risk-free rate
        sigma (float): volatility
        option_type (str): "call" or "put"

    Returns:
        option_price (float): price of the option
    """

    d1 = (np.log(S/K) + (r + sigma**2/2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        # option_price = K * np.exp(-r * T) * norm.cdf(-d2)
        option_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return option_price

#delta stratergy
def delta_stratergy(S, K, r, T, sigma_guess, N, option_type):
    '''
    Calculates the self-financing strategy for a portfolio consisting of a European call and put option on a weekly basis.
    Args:
        S (float): The current stock price.
        K (float): The strike price of the options.
        r (float): The risk-free interest rate.
        T (float): The time to maturity of the options in years.
        vol (float): The implied volatility of the options.
    Returns:
        delta (list): A list of the values of delta for the self-financing strategy on a weekly basis.
    '''

    try:
      option_price, _ = binomial_model(S, K, T, r, sigma_guess, N, option_type)
      Q = 0
      vol = implied_volatility(option_price, S, K, T, r, Q, flag="p")
      print("VOL: ", vol)

      t = T

      # Calculate d1 and d2
      d1 = (math.log(S/K) + (r + 0.5*vol**2)*(T)) / (vol*math.sqrt(T))
      d2 = d1 - vol*math.sqrt(T)

      # Calculate the call and put option prices
      call_price = S*norm.cdf(d1) - K*math.exp(-r*(T))*norm.cdf(d2)
      put_price = K*math.exp(-r*(T))*norm.cdf(-d2) - S*norm.cdf(-d1)

      # Calculate delta
      delta_value = (call_price - put_price) / (2*S*vol**2*(T))

      # Append the value of delta to the list
    except:
      delta_value = "NaN"
    return delta_value

Variables

In [ ]:
#define variables
R = 0.007

#calculating sigma
returns = stocks_clean.pct_change().dropna()
sigma = np.std(returns, axis=0)
print(sigma)

AAPL    0.021506
AMZN    0.030112
TSLA    0.041375
dtype: float64


In [ ]:
end_date = '2023-04-28'
start_date = '2022-04-27'
tickers = ["AAPL", "AMZN", "TSLA"]

# Download stock data
stocks_weekly = yf.download(tickers, start=start_date, end=end_date, interval="1wk")
stocks_weekly = stocks_weekly["Adj Close"]
K_AAPL = [(i + 2) for i in stocks_weekly["AAPL"]]
K_AMZN = [(i + 2) for i in stocks_weekly["AMZN"]]
K_TSLA = [(i + 2) for i in stocks_weekly["TSLA"]]
T_ = [1, 8, 15]

[*********************100%***********************]  3 of 3 completed


Use (CRR) Binomial model (2 Steps to 10 Steps) to price the corresponding European put option and American put option with different strike price and expiry date.

Expiry Date: April 28th 2023

Stock: AAPL

Binomial Model

In [ ]:
exp_28_stock_aapl = {}
for sample, row in enumerate(put_option_data_AAPL.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 1
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_28_stock_aapl[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], N=n, option_type=option_type)
      exp_28_stock_aapl[option_type].append([option_price, stock_price_])
      print("Steps: {} | Current Stock Price : {} | Strike Price : {:.4f} | Actual option price : {:.4f} | Option Price: {:.4f} | Future Stock Price : {:.4f}".format(n, stock_price, strike_price, row[5], option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock Price : 163.7600
Steps: 3 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock Price : 163.7600
Steps: 4 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock Price : 163.7600
Steps: 5 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock Price : 163.7600
Steps: 6 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock Price : 163.7600
Steps: 7 | Current Stock Price : 163.75999450683594 | Strike Price : 50.0000 | Actual option price : 0.0100 | Option Price: 0.0000 | Future Stock P

Black Scholes Model

In [ ]:
exp_28_stock_aapl_bs = []
for iter, row in enumerate(put_option_data_AAPL.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 1
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_28_stock_aapl_bs.append(option_price)

Sample Index: 0 | Option Price : 0.0
Sample Index: 1 | Option Price : 0.0
Sample Index: 2 | Option Price : 6.1318009111004844e-235
Sample Index: 3 | Option Price : 4.64257989442351e-177
Sample Index: 4 | Option Price : 7.290906756343617e-141
Sample Index: 5 | Option Price : 1.9309156687124226e-125
Sample Index: 6 | Option Price : 1.3632968755426379e-106
Sample Index: 7 | Option Price : 6.9379587019959435e-84
Sample Index: 8 | Option Price : 9.425412990379868e-67
Sample Index: 9 | Option Price : 1.9245465607498916e-50
Sample Index: 10 | Option Price : 5.896134574347018e-41
Sample Index: 11 | Option Price : 1.326664857230456e-33
Sample Index: 12 | Option Price : 4.746134835321975e-25
Sample Index: 13 | Option Price : 8.618192623730712e-22
Sample Index: 14 | Option Price : 1.1974499814100642e-18
Sample Index: 15 | Option Price : 2.2087007158426064e-17
Sample Index: 16 | Option Price : 3.6630779484154844e-16
Sample Index: 17 | Option Price : 4.745494110785684e-18
Sample Index: 18 | Option 

Self Financing Stratergy

In [ ]:
for T in T_:
  print("T : {}".format(T))
  for index, sample in enumerate(stocks_weekly["AAPL"]):
    stock_price = sample
    K = K_AAPL[index]
    T = 1
    for option_type in ["American", "European"]:
      print("Option Type: {}".format(option_type))
      for n in range(2, 11):
        delta_initial = delta_stratergy(S=stock_price, K=strike_price, T = time_period, r = R, sigma_guess = sigma[0], N=n, option_type=option_type)
        print("Steps: {} | Current Stock Price : {} | Delta : {} | Strike Price : {:.4f} | Actual option price : {:.4f} | Option Price: {:.4f} | Future Stock Price : {:.4f}".format(n, stock_price, delta_initial, strike_price, row[5], option_price, stock_price_))

Option Type: American
VOL:  0.2718777351749825
Steps: 2 | Current Stock Price : 156.7007293701172 | Delta : -3.309181576749097 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.2718777351749825
Steps: 3 | Current Stock Price : 156.7007293701172 | Delta : -3.309181576749097 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.2718777351749825
Steps: 4 | Current Stock Price : 156.7007293701172 | Delta : -3.309181576749097 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.2718777351749825
Steps: 5 | Current Stock Price : 156.7007293701172 | Delta : -3.309181576749097 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.2718777351749825
Steps: 6 | Current Stock Price : 156.7007293701172 | Delta : -3.3091815767490

Stock: AMZN

Binomial Model

In [ ]:
exp_28_stock_amzn = {}
for sample, row in enumerate(put_option_data_AMZN.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 1
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_28_stock_amzn[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], N=n, option_type=option_type)
      exp_28_stock_amzn[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes Model


In [ ]:
exp_28_stock_amzn_bs = []
for iter, row in enumerate(put_option_data_AMZN.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 1
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_28_stock_amzn_bs.append(option_price)

Sample Index: 0 | Option Price : 0.0
Sample Index: 1 | Option Price : 1.790290632298521e-245
Sample Index: 2 | Option Price : 2.297841041175491e-118
Sample Index: 3 | Option Price : 7.730021280568572e-89
Sample Index: 4 | Option Price : 2.3331600229345676e-70
Sample Index: 5 | Option Price : 1.7480664403877645e-62
Sample Index: 6 | Option Price : 7.294056379386324e-53
Sample Index: 7 | Option Price : 2.8119760844009127e-41
Sample Index: 8 | Option Price : 1.5571267762658856e-32
Sample Index: 9 | Option Price : 3.27093792917775e-24
Sample Index: 10 | Option Price : 2.2795255005290703e-19
Sample Index: 11 | Option Price : 1.2958710746943277e-15
Sample Index: 12 | Option Price : 3.0075417065863156e-11
Sample Index: 13 | Option Price : 1.3794804188253534e-09
Sample Index: 14 | Option Price : 5.5168486324464425e-08
Sample Index: 15 | Option Price : 2.442000545008418e-07
Sample Index: 16 | Option Price : 1.0237786852516528e-06
Sample Index: 17 | Option Price : 1.1240702318196224e-07
Sample I

Self Financing Stratergy

In [ ]:
for T in T_:
  print("T : {}".format(T))
  for index, sample in enumerate(stocks_weekly["AMZN"]):
    stock_price = sample
    K = K_AMZN[index]
    T = 1
    for option_type in ["American", "European"]:
      print("Option Type: {}".format(option_type))
      for n in range(2, 11):
        delta_initial = delta_stratergy(S=stock_price, K=strike_price, T = time_period, r = R, sigma_guess = sigma[0], N=n, option_type=option_type)
        print("Steps: {} | Current Stock Price : {} | Delta : {} | Strike Price : {:.4f} | Actual option price : {:.4f} | Option Price: {:.4f} | Future Stock Price : {:.4f}".format(n, stock_price, delta_initial, strike_price, row[5], option_price, stock_price_))

Streaming output truncated to the last 5000 lines.
VOL:  0.4678022951093728
Steps: 8 | Current Stock Price : 109.55999755859375 | Delta : -2.5817641001681317 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.4678022951093728
Steps: 9 | Current Stock Price : 109.55999755859375 | Delta : -2.5817641001681317 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.4678022951093728
Steps: 10 | Current Stock Price : 109.55999755859375 | Delta : -2.5817641001681317 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
Option Type: European
Steps: 2 | Current Stock Price : 109.55999755859375 | Delta : NaN | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.09827544510659213
Steps: 3 | Current Stock Price : 109.55999755859375 | Del

Stocks: TSLA

Binomial Model

In [ ]:
exp_28_stock_tsla = {}
for sample, row in enumerate(put_option_data_TSLA.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 1
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_28_stock_tsla[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], N=n, option_type=option_type)
      exp_28_stock_tsla[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes Model

In [ ]:
exp_28_stock_tsla_bs = []
for iter, row in enumerate(put_option_data_TSLA.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 1
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_28_stock_tsla_bs.append(option_price)

Self Financing Stratergy

In [ ]:
for T in T_:
  print("T : {}".format(T))
  for index, sample in enumerate(stocks_weekly["TSLA"]):
    stock_price = sample
    K = K_TSLA[index]
    T = 1
    for option_type in ["American", "European"]:
      print("Option Type: {}".format(option_type))
      for n in range(2, 11):
        delta_initial = delta_stratergy(S=stock_price, K=strike_price, T = time_period, r = R, sigma_guess = sigma[0], N=n, option_type=option_type)
        print("Steps: {} | Current Stock Price : {} | Delta : {} | Strike Price : {:.4f} | Actual option price : {:.4f} | Option Price: {:.4f} | Future Stock Price : {:.4f}".format(n, stock_price, delta_initial, strike_price, row[5], option_price, stock_price_))

Streaming output truncated to the last 5000 lines.
VOL:  0.02032302916072847
Steps: 10 | Current Stock Price : 250.76333618164062 | Delta : 84.01232916221747 | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
Option Type: European
VOL:  0.0
Steps: 2 | Current Stock Price : 250.76333618164062 | Delta : NaN | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.0
Steps: 3 | Current Stock Price : 250.76333618164062 | Delta : NaN | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.0
Steps: 4 | Current Stock Price : 250.76333618164062 | Delta : NaN | Strike Price : 235.0000 | Actual option price : 68.6500 | Option Price: 71.4848 | Future Stock Price : 164.9000
VOL:  0.0
Steps: 5 | Current Stock Price : 250.76333618164062 | Delta : NaN | Strike Price : 235.0000 | Actual option price : 68.6500 | 

Expiry Date: May 5th 2023

Stocks: AAPL

Binomial Model

In [ ]:
exp_5_stock_aapl = {}
for sample, row in enumerate(put_option_data_AAPL_2.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 8
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_5_stock_aapl[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], N=n, option_type=option_type)
      exp_5_stock_aapl[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 3 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 4 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 5 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 6 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 7 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 8 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 9 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 10 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 3 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 4 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 5 | Option Price: 0.0 | Future Stock Price : 163.75999450683594
Steps: 6 | Optio

Black Scholes Model

In [ ]:
exp_5_stock_aapl_bs = []
for iter, row in enumerate(put_option_data_AAPL.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 8
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_5_stock_aapl_bs.append(option_price)

Sample Index: 0 | Option Price : 4.5065003128481685e-91
Sample Index: 1 | Option Price : 2.4953603094940776e-66
Sample Index: 2 | Option Price : 2.996565158065588e-33
Sample Index: 3 | Option Price : 1.9895171148220927e-25
Sample Index: 4 | Option Price : 1.8650782895065424e-20
Sample Index: 5 | Option Price : 2.5579928168457646e-18
Sample Index: 6 | Option Price : 1.0787134159377342e-15
Sample Index: 7 | Option Price : 1.6810128524696834e-12
Sample Index: 8 | Option Price : 4.712900367805178e-10
Sample Index: 9 | Option Price : 1.1099576005125973e-07
Sample Index: 10 | Option Price : 2.8731066068640944e-06
Sample Index: 11 | Option Price : 3.770604029393466e-05
Sample Index: 12 | Option Price : 0.0008058447431639814
Sample Index: 13 | Option Price : 0.002649231418204766
Sample Index: 14 | Option Price : 0.008527536234993307
Sample Index: 15 | Option Price : 0.013803703302583245
Sample Index: 16 | Option Price : 0.022059527285304577
Sample Index: 17 | Option Price : 0.01086471933427198

Stocks: AMZN

Binomial Model

In [ ]:
exp_5_stock_amzn = {}
for sample, row in enumerate(put_option_data_AMZN_2.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 8
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_5_stock_amzn[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], N=n, option_type=option_type)
      exp_5_stock_amzn[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes Model

In [ ]:
exp_5_stock_amzn_bs = []
for iter, row in enumerate(put_option_data_AMZN.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 8
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_5_stock_amzn_bs.append(option_price)

Sample Index: 0 | Option Price : 1.3215948499005975e-21
Sample Index: 1 | Option Price : 5.590874997208966e-14
Sample Index: 2 | Option Price : 1.9287142707505085e-11
Sample Index: 3 | Option Price : 5.429105046792822e-09
Sample Index: 4 | Option Price : 2.5562267980164267e-06
Sample Index: 5 | Option Price : 2.9388512416801845e-07
Sample Index: 6 | Option Price : 6.270938718396145e-05
Sample Index: 7 | Option Price : 0.00014028172260638472
Sample Index: 8 | Option Price : 8.211173079092956e-05
Sample Index: 9 | Option Price : 1.1440932064303965e-05
Sample Index: 10 | Option Price : 2.609770494904861e-05
Sample Index: 11 | Option Price : 5.7557864784425606e-05
Sample Index: 12 | Option Price : 0.0001228829401017527
Sample Index: 13 | Option Price : 0.00025425153871874083
Sample Index: 14 | Option Price : 0.0005103873386683118
Sample Index: 15 | Option Price : 0.0009950838330288266
Sample Index: 16 | Option Price : 0.0018861802173789502
Sample Index: 17 | Option Price : 0.00347930794816

Stocks: TSLA

Binomial Model

In [ ]:
exp_5_stock_tsla = {}
for sample, row in enumerate(put_option_data_TSLA_2.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 8
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_5_stock_tsla[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], N=n, option_type=option_type)
      exp_5_stock_tsla[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes

In [ ]:
exp_5_stock_tsla_bs = []
for iter, row in enumerate(put_option_data_TSLA.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 8
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_5_stock_tsla_bs.append(option_price)

Sample Index: 0 | Option Price : 3.099100398048539e-73
Sample Index: 1 | Option Price : 1.5360725246953645e-40
Sample Index: 2 | Option Price : 3.8411834056664825e-23
Sample Index: 3 | Option Price : 1.3225315134857596e-16
Sample Index: 4 | Option Price : 3.008505625598757e-11
Sample Index: 5 | Option Price : 3.012243754241894e-09
Sample Index: 6 | Option Price : 1.8965886566910788e-06
Sample Index: 7 | Option Price : 9.859157534765745e-05
Sample Index: 8 | Option Price : 0.001989215126764962
Sample Index: 9 | Option Price : 0.005199266767995943
Sample Index: 10 | Option Price : 0.025898733065332788
Sample Index: 11 | Option Price : 0.1045135007355972
Sample Index: 12 | Option Price : 0.35036519788497195
Sample Index: 13 | Option Price : 0.9971100497687557
Sample Index: 14 | Option Price : 1.7401802446489332
Sample Index: 15 | Option Price : 2.072340954339382
Sample Index: 16 | Option Price : 2.4549797244110514
Sample Index: 17 | Option Price : 2.8934043275935357
Sample Index: 18 | Opt

Expiry Date: May 12th 2023

Stocks: AAPL

In [ ]:
exp_12_stock_aapl = {}
for sample, row in enumerate(put_option_data_AAPL_3.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 15
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_12_stock_aapl[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], N=n, option_type=option_type)
      exp_12_stock_aapl[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes

In [ ]:
exp_12_stock_aapl_bs = []
for iter, row in enumerate(put_option_data_AAPL.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AAPL"][launch_date]
  time_period = 15
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[0], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_12_stock_aapl_bs.append(option_price)

Sample Index: 0 | Option Price : 1.4048971694707677e-52
Sample Index: 1 | Option Price : 8.929333130814845e-39
Sample Index: 2 | Option Price : 4.26612835013248e-20
Sample Index: 3 | Option Price : 1.3042837436559178e-15
Sample Index: 4 | Option Price : 1.0131166167404926e-12
Sample Index: 5 | Option Price : 1.8071391194979118e-11
Sample Index: 6 | Option Price : 6.31034921136595e-10
Sample Index: 7 | Option Price : 4.9084704065307945e-08
Sample Index: 8 | Option Price : 1.4403921960745902e-06
Sample Index: 9 | Option Price : 3.97553003437403e-05
Sample Index: 10 | Option Price : 0.00029680676376007547
Sample Index: 11 | Option Price : 0.0014893639427052106
Sample Index: 12 | Option Price : 0.010422405367265021
Sample Index: 13 | Option Price : 0.022402348470877537
Sample Index: 14 | Option Price : 0.047893485035524624
Sample Index: 15 | Option Price : 0.06577249674816926
Sample Index: 16 | Option Price : 0.08972075783678542
Sample Index: 17 | Option Price : 0.05651723813163069
Sample 

Stocks: AMZN

In [ ]:
exp_12_stock_amzn = {}
for sample, row in enumerate(put_option_data_AMZN_3.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 15
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_12_stock_amzn[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], N=n, option_type=option_type)
      exp_12_stock_amzn[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes Model

In [ ]:
exp_12_stock_amzn_bs = []
for iter, row in enumerate(put_option_data_AMZN.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["AMZN"][launch_date]
  time_period = 15
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[1], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_12_stock_amzn_bs.append(option_price)

Sample Index: 0 | Option Price : 6.597964927360754e-13
Sample Index: 1 | Option Price : 1.5285486485459897e-08
Sample Index: 2 | Option Price : 4.6204923504552904e-07
Sample Index: 3 | Option Price : 1.275894030921616e-05
Sample Index: 4 | Option Price : 0.000498016334050954
Sample Index: 5 | Option Price : 0.00013863298437692855
Sample Index: 6 | Option Price : 0.0034351285526606247
Sample Index: 7 | Option Price : 0.005624148099721957
Sample Index: 8 | Option Price : 0.004087335116514604
Sample Index: 9 | Option Price : 0.0012541230342202516
Sample Index: 10 | Option Price : 0.002068752876671485
Sample Index: 11 | Option Price : 0.003349640540170623
Sample Index: 12 | Option Price : 0.005327157395064623
Sample Index: 13 | Option Price : 0.008326735866164803
Sample Index: 14 | Option Price : 0.0127996611969469
Sample Index: 15 | Option Price : 0.01936056490842501
Sample Index: 16 | Option Price : 0.028831978663396624
Sample Index: 17 | Option Price : 0.042295960624373165
Sample Index:

Stocks: TSLA

Binomial Model

In [ ]:
exp_12_stock_tsla = {}
for sample, row in enumerate(put_option_data_TSLA_3.values):
  print("Sample Index: {}".format(sample))
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 15
  for option_type in ["American", "European"]:
    print("Option Type: {}".format(option_type))
    exp_12_stock_tsla[option_type] = []
    for n in range(2, 11):
      option_price, stock_price_ = binomial_model(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], N=n, option_type=option_type)
      exp_12_stock_tsla[option_type].append([option_price, stock_price])
      print("Steps: {} | Option Price: {} | Future Stock Price : {}".format(n, option_price, stock_price_))

Sample Index: 0
Option Type: American
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 10 | Option Price: 0.0 | Future Stock Price : 0.0
Option Type: European
Steps: 2 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 3 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 4 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 5 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 6 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 7 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 8 | Option Price: 0.0 | Future Stock Price : 0.0
Steps: 9 | Option Price: 0.0 | Future Stock

Black Scholes Model

In [ ]:
exp_12_stock_tsla_bs = []
for iter, row in enumerate(put_option_data_TSLA.values):
  strike_price = row[2]
  launch_date = row[1][:10]
  stock_price = stocks_clean["TSLA"][launch_date]
  time_period = 15
  option_price = black_scholes(S=stock_price, K=strike_price, T = time_period, r = R, sigma = sigma[2], option_type="put")
  print("Sample Index: {} | Option Price : {}".format(iter, option_price))
  exp_12_stock_tsla_bs.append(option_price)